In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers
!pip3 install accelerate
!pip3 install diffusers
!pip3 install emoji

In [ ]:
%%capture
import os
import shutil
import torch
import json
import re
import requests
import emoji
import random
import difflib
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from itertools import combinations
from diffusers import StableDiffusionPipeline
from transformers import CLIPTextModelWithProjection
from transformers import BlipProcessor, BlipForQuestionAnswering
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = 'cuda'
model_id = "runwayml/stable-diffusion-v1-5"
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker = None)
pipe = pipe.to(device)
pipe.enable_attention_slicing()

tokenizer = pipe.tokenizer
tokenizer.pad_token = tokenizer.eos_token

text_model = CLIPTextModelWithProjection.from_pretrained(model_id, subfolder="text_encoder")
text_model.to(device)

image_processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
image_model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.92it/s]


In [ ]:
def pil_to_latent(img):
    image = pipe.image_processor.preprocess(img).to(pipe.device).half()
    image_latents = (
        pipe.vae.encode(image).latent_dist.sample() * pipe.vae.config.scaling_factor
    )
    return image_latents

In [ ]:
def get_text_embeddings(text):
    token_ids = tokenizer(text, max_length=77, truncation=True, padding="max_length",return_tensors="pt").input_ids
    return text_model(input_ids=token_ids.to(device)).text_embeds

def calculate_text_embedding_shift(text_a, text_b):
    text_a_emb = get_text_embeddings(text_a)
    text_b_emb = get_text_embeddings(text_b)
    cos_sim = cos(text_a_emb.view(-1), text_b_emb.view(-1))
    return cos_sim.item()

In [ ]:
def set_seed(seed):
  gen = torch.Generator(device=device)
  random.seed(seed)
  np.random.seed(seed)
  torch.cuda.manual_seed(seed)
  torch.manual_seed(seed)
  return gen.manual_seed(seed)

def generate_image(prompt, seed, filename=None, save=False):
  gen = set_seed(seed)
  image = pipe(prompt, num_inference_steps=20, generator=gen).images[0]
  plt.imshow(image)
  plt.axis('off')
  if save:
    image.save(filename+".jpg")
  plt.show()
  return image

def generate_image_no_save(prompt, seed):
    gen = set_seed(seed)
    image = pipe(prompt, num_inference_steps=20, generator=gen).images[0]
    return image

def generate_image_no_show(prompt, seed, filename=None, save=False):
  gen = set_seed(seed)
  image = pipe(prompt, num_inference_steps=20, generator=gen).images[0]
  if save:
    image.save(filename+".jpg")
  return image

In [ ]:
def find_all_combinations(ids, placeholder):

    combinations_list = []
    for n in range(1, len(ids)): #len(ids)
        for comb in combinations(range(len(ids)), n):
            new_list = ids[:]
            for index in comb:
                new_list[index] = placeholder
            if placeholder in new_list:
                combinations_list.append(new_list)

    return combinations_list

In [ ]:
def calculate_image_embedding_shift(img_a, img_b):
    img_a_emb = pil_to_latent(img_a)
    img_b_emb = pil_to_latent(img_b)
    sim = cos(img_a_emb.view(-1), img_b_emb.view(-1))
    return sim.item()

In [ ]:
def check_attack_success(img, text):
    question = "Does the image match the caption: \"{}\"? Answer only Yes or No?"
    template = question.format(text)
    prompt = f"[INST] <image>\n{template} [/INST]"

    inputs = image_processor(text=prompt, images=img, return_tensors="pt").to(device)
    out = image_model.generate(**inputs, max_new_tokens=100)
    #print(image_processor.decode(out[0], skip_special_tokens=True))
    gen = out[:, inputs.input_ids.shape[1]:]
    answer = image_processor.decode(gen[0], skip_special_tokens=True).strip()

    if answer == "Yes":
        return 1
    else:
        return 0

In [ ]:
def find_target_word(input, target):
  words_A = re.split(r'\s+|\.', input)
  words_B = re.split(r'\s+|\.', target)
  differ = difflib.ndiff(words_A, words_B)
  output = [word for word in differ if word.startswith('+ ')][0][2:]
  return output

In [ ]:
def make_directory(dirs_path):
    if os.path.exists(dirs_path) and os.path.isdir(dirs_path):
        try:
            shutil.rmtree(dirs_path)
            print(f"Directory '{dirs_path}' has been removed")
        except OSError as e:
            print(f"Error: {dirs_path} : {e.strerror}")
    os.makedirs(dirs_path)

In [ ]:
def input_prompts(in_text, tar_text, suffix, seed):
    input_text = in_text
    target_text = tar_text
    suffix_prompt = emoji.emojize(suffix)

    print(suffix_prompt)

    tokens = tokenizer.tokenize(suffix_prompt)
    id_list = tokenizer.convert_tokens_to_ids(tokens)
    print(len(id_list))

    target_word = find_target_word(input_text, target_text)
    input_word  = find_target_word(target_text, input_text)
    print(input_word, " ", target_word)

    boundary = calculate_text_embedding_shift(input_word, target_word)
    print("Cosine similarity between Input word and Target Word:", boundary)

    dir_path = input_text.replace(" ","_").replace(".","_")+"___"+target_text.replace(" ","_").replace(".","_")+"/"
    dir_path = "./" + dir_path
    make_directory(dir_path)

    input_img = generate_image(input_text, seed, dir_path+"input_{}".format(seed), True)
    target_img = generate_image(target_text, seed, dir_path+"target", True)
    suffix_img = generate_image(input_text+suffix_prompt, seed, dir_path+"generated", True)

    inp_to_tar = calculate_image_embedding_shift(input_img, target_img)
    suffix_to_tar = calculate_image_embedding_shift(suffix_img, target_img)
    suffix_to_in = calculate_image_embedding_shift(suffix_img, input_img)

    origin_text_sim = calculate_text_embedding_shift(input_text, target_text)
    print("Cosine similarity between input and target text: ", origin_text_sim)

    generated_text_sim = calculate_text_embedding_shift(input_text+suffix_prompt, target_text)
    print("Cosine similarity between input+suffix and target text: ", generated_text_sim)

    in_text_sim = calculate_text_embedding_shift(input_text+suffix_prompt, input_text)
    print("Cosine similarity between input+suffix and input text: ", in_text_sim)

    print("Cosine similarity between input and target image: ", inp_to_tar)
    print("Cosine similarity between generated and target image: ",suffix_to_tar)
    print("Cosine similarity between generated and input image: ",suffix_to_in)

    return input_text, target_text, suffix_prompt, id_list, dir_path, input_word, target_word, input_img, target_img

In [ ]:
def extract_unsuccessful_combinations(input_text, target_text, input_img, target_img, id_list, dir_path, seed, placeholder):

    candidates = find_all_combinations(id_list, placeholder)

    cand_text_list = []
    for comb in candidates:
        sfx = tokenizer.decode(comb, skip_special_tokens = False)
        cand_text = input_text +" "+ sfx
        cand_text_list.append(cand_text)

    comb_id_list, comb_text_list, comb_img_list, all_image = [], [], [], []
    for idx, txt in enumerate(cand_text_list):

        if idx%100 == 0:
            print(f"Sample {idx}")

        image = generate_image_no_save(txt, seed)
        all_image.append(image)
        ans = check_attack_success(image, target_text)
        if ans == 0:
            comb_id_list.append(candidates[idx])
            comb_text_list.append(cand_text_list[idx])
            comb_img_list.append(image)
        else:
            continue

    path = dir_path+"unsuccessful_combinations/"
    make_directory(path)
    for ppos in range(len(comb_text_list)):
        tx = comb_text_list[ppos]
        im =  comb_img_list [ppos]
        im.save(path+f"{ppos}.jpg")

    candidate_list, text_shift_list, image_shift_list = [], [], []
    for idx, cand in enumerate(comb_id_list):
        sfx = tokenizer.decode(cand, skip_special_tokens=False)
        cand_text = input_text+sfx
        cand_img = comb_img_list[idx]

        text_sim_a = calculate_text_embedding_shift(cand_text, input_text)
        text_sim_b = calculate_text_embedding_shift(cand_text, target_text)

        image_sim_a = calculate_image_embedding_shift(cand_img, input_img)
        image_sim_b = calculate_image_embedding_shift(cand_img, target_img)

        candidate_list.append(sfx)
        text_shift_list.append(text_sim_b - text_sim_a)
        image_shift_list.append(image_sim_b - image_sim_a)

    df = pd.DataFrame()
    df['suffix_comb'] = candidate_list
    df['text_shift'] = text_shift_list
    df['img_shift'] = image_shift_list
    df.to_csv(path+"suffix_combination.csv", index=False)

    return candidates, comb_id_list, comb_img_list, all_image

In [ ]:
def find_critical_tokens(input_text, target_text, id_list, comb_id_list, comb_img_list, all_image, candidates, placeholder, seed):

    critical_tokens_id, non_critical_tokens_id = [], []
    critical_token_comb_idx = -1
    critical_tok_length = -1
    for i in range(len(comb_id_list)):
        critical_token_length = comb_id_list[i].count(placeholder)

        critical_token_comb_idx = i
        non_crit = comb_id_list[i]
        crit = [placeholder]*len(id_list)
        for idx, x in enumerate(id_list):
            if x not in non_crit:
                crit[idx] = x

        #crit_img = all_image[crit_idx_in_comb] #generate_image_no_show(crit_text, seed)
        crit_text = input_text + tokenizer.decode(crit, skip_special_tokens=False)
        crit_img = generate_image_no_show(crit_text, seed)

        in_succ = check_attack_success(crit_img, input_text)
        crit_succ = check_attack_success(crit_img, target_text)

        if in_succ == 0 and crit_succ == 1:
            critical_tok_length = critical_token_length
            critical_tokens_id = crit.copy()
            non_critical_tokens_id = [x for x in non_crit if x != placeholder]
            break

    print(f"Inside function: {critical_tok_length}, and {critical_tokens_id}, and {non_critical_tokens_id}")
    return critical_token_comb_idx, critical_tok_length, critical_tokens_id, non_critical_tokens_id

In [ ]:
def non_critical_placeholder_check(input_text, target_text, input_img, target_img, non_critical_checklist, non_critical_tokens_id, dir_path, seed):

    path = dir_path+"non_critic_placeholder/"
    make_directory(path)
    cnt = 0
    candidate_list, text_shift_list, image_shift_list = [], [], []

    for idx, tok_ids in enumerate(non_critical_checklist):
        sfx = tokenizer.decode(tok_ids, skip_special_tokens=False)
        cand_text = input_text + sfx
        cand_img = generate_image_no_show(cand_text, seed, path+f"{idx}", True)

        text_sim_a = calculate_text_embedding_shift(cand_text, input_text)
        text_sim_b = calculate_text_embedding_shift(cand_text, target_text)

        image_sim_a = calculate_image_embedding_shift(cand_img, input_img)
        image_sim_b = calculate_image_embedding_shift(cand_img, target_img)

        candidate_list.append(sfx)
        text_shift_list.append(text_sim_b - text_sim_a)
        image_shift_list.append(image_sim_b - image_sim_a)

        in_succ = check_attack_success(cand_img, input_text)
        tar_succ = check_attack_success(cand_img, target_text)
        if in_succ == 0 and tar_succ == 1:
            cnt += 1

    text_shift = "higher (as expected)"
    for x in text_shift_list:
        if x < 0:
            text_shift = "lower"
            break

    image_shift = "higher (as expected)"
    for y in image_shift_list:
        if y < 0:
            image_shift = "lower"
            break

    match = False
    if len(non_critical_checklist) == cnt and len(non_critical_checklist) > 0:
        match = True

    df = pd.DataFrame()
    df['candidate_suffix'] = candidate_list
    df['text_shift'] = text_shift_list
    df['img_shift'] = image_shift_list
    df.to_csv(path+"suffix_evaluation.csv", index=False)

    non_crit_tokens = []
    if len(non_critical_tokens_id) > 0:
        non_crit_tokens = tokenizer.convert_ids_to_tokens(non_critical_tokens_id, skip_special_tokens=True)

    diction = {
        "tokens": non_crit_tokens,
        "text_shift": text_shift,
        "image_shift": image_shift,
        "images_matched_target": cnt,
        "simply_remove": match
    }
    json_object = json.dumps(diction, indent=4)
    with open(path+"info.json", "w") as outfile:
        outfile.write(json_object)


In [ ]:
def non_critical_removal_check(input_text, target_text, input_img, target_img, non_critical_checklist, non_critical_tokens_id, dir_path, seed):

    path = dir_path+"non_critic_removed/"
    make_directory(path)
    dnt = 0
    candidate_list, text_shift_list, image_shift_list = [], [], []

    for idx, tok_ids in enumerate(non_critical_checklist):
        sfx = tokenizer.decode(tok_ids, skip_special_tokens=True)
        cand_text = input_text + sfx
        cand_img = generate_image_no_show(cand_text, seed, path+f"{idx}", True)

        text_sim_a = calculate_text_embedding_shift(cand_text, input_text)
        text_sim_b = calculate_text_embedding_shift(cand_text, target_text)

        image_sim_a = calculate_image_embedding_shift(cand_img, input_img)
        image_sim_b = calculate_image_embedding_shift(cand_img, target_img)

        candidate_list.append(sfx)
        text_shift_list.append(text_sim_b - text_sim_a)
        image_shift_list.append(image_sim_b - image_sim_a)

        in_succ = check_attack_success(cand_img, input_text)
        tar_succ = check_attack_success(cand_img, target_text)
        if in_succ == 0 and tar_succ == 1:
            dnt += 1

    text_shift = "higher (as expected)"
    for x in text_shift_list:
        if x < 0:
            text_shift = "lower"
            break

    image_shift = "higher (as expected)"
    for y in image_shift_list:
        if y < 0:
            image_shift = "lower"
            break

    match = False
    if len(non_critical_checklist) == dnt and len(non_critical_checklist) > 0:
        match = True

    df = pd.DataFrame()
    df['candidate_suffix'] = candidate_list
    df['text_shift'] = text_shift_list
    df['img_shift'] = image_shift_list
    df.to_csv(path+"suffix_evaluation.csv", index=False)

    non_crit_tokens = []
    if len(non_critical_tokens_id) > 0:
        non_crit_tokens = tokenizer.convert_ids_to_tokens(non_critical_tokens_id, skip_special_tokens=True)

    diction = {
        "tokens": non_crit_tokens,
        "text_shift": text_shift,
        "image_shift": image_shift,
        "images_matched_target": dnt,
        "simply_remove": match
    }
    json_object = json.dumps(diction, indent=4)
    with open(path+"info.json", "w") as outfile:
        outfile.write(json_object)

    return match

In [ ]:
def show_stats(input_word, target_word, id_list, comb_id_list, placeholder):
    tokens_li = id_list
    lists_of_tokens =  comb_id_list
    replacement_count = {token: 0 for token in tokens_li}
    for token_list in lists_of_tokens:
        for i, token in enumerate(token_list):
            if token == placeholder:
                original_token = tokens_li[i]
                replacement_count[original_token] += 1

    freq_info = []
    zero_occur = []
    for token, count in replacement_count.items():
        freq_info.append(f"{tokenizer.convert_ids_to_tokens(token)}' was replaced by 49407 {count} times.")
        if count == 0:
            zero_occur.append(token)
    print(replacement_count)

    ix = tokenizer.tokenize(input_word)[0]
    tx = tokenizer.tokenize(target_word)[0]
    critical_sim_list = [[], [], []]
    for idx, tok_id in enumerate(id_list):
        tok = tokenizer.convert_ids_to_tokens(tok_id)
        sim = calculate_text_embedding_shift(tok, tx)
        critical_sim_list[0].append(tok)
        critical_sim_list[1].append(sim)

    for idx, tok_id in enumerate(id_list):
        tok = tokenizer.convert_ids_to_tokens(tok_id)
        sim = calculate_text_embedding_shift(tok, ix)
        critical_sim_list[2].append(sim)

    ct_closeto_target = ["No"]*len(critical_sim_list[0])
    for i in range(len(critical_sim_list[0])):
        if critical_sim_list[1][i] > critical_sim_list[2][i]:
            ct_closeto_target[i] = "Yes"

    return freq_info, critical_sim_list, ct_closeto_target, zero_occur

In [ ]:
def main_job(inp, tar, suf, seed, placeholder):

    in_txt, tar_txt, suf_txt, id_list, dir_path, in_word, tar_word, in_img, tar_img = input_prompts(inp, tar, suf, seed)

    cand_id_list, comb_id_list, comb_img_list, all_img = extract_unsuccessful_combinations(in_txt, tar_txt, in_img, tar_img, id_list, dir_path, seed, placeholder)

    freq_info, critical_sim_list, ct_closeto_target, zero_count = show_stats(in_word, tar_word, id_list, comb_id_list, placeholder)

    crit_idx, crit_len, crit_id, non_crit_id = find_critical_tokens(in_txt, tar_txt, id_list, comb_id_list, comb_img_list, all_img, cand_id_list, placeholder, seed)

    if len(zero_count) > 0:
        if len(crit_id) == 0:
            crit_id = [x for x in id_list if x not in zero_count]

        if len(non_crit_id) > 0:
            for itm in zero_count:
                if itm in non_crit_id:
                    continue
                else:
                    non_crit_id.append(itm)
        else:
            for itm in zero_count:
                non_crit_id.append(itm)

    if len(non_crit_id) == 0:
        dictonary = {
        "input_text": in_txt,
        "target_text": tar_txt,
        "initial_suffix": emoji.demojize(suf_txt),
        "frequency": freq_info,
        "critical_tokens": crit_id,
        "non_critical_tokens": [],
        "CT_sim_target": critical_sim_list[1],
        "CT_sim_input": critical_sim_list[2],
        "CT_closeto_target": ct_closeto_target
        }
        json_object = json.dumps(dictonary, indent=4)
        with open(dir_path+"info.json", "w") as outfile:
            outfile.write(json_object)
        print("0 Non-crtical Tokens!")

    non_critical_checklist = []
    indices_to_replace = [i for i, x in enumerate(id_list) if x in non_crit_id]
    for r in range(1, len(indices_to_replace)+1):
        for comb in combinations(indices_to_replace, r):
            new_list = id_list[:]
            for index in comb:
                new_list[index] = placeholder
            non_critical_checklist.append(new_list)

    match = False
    if len(non_crit_id) < len(id_list):
        non_critical_placeholder_check(in_txt, tar_txt, in_img, tar_img, non_critical_checklist, non_crit_id, dir_path, seed)
        match = non_critical_removal_check(in_txt, tar_txt, in_img, tar_img, non_critical_checklist, non_crit_id, dir_path, seed)

    crit_tokens, non_crit_tokens = [], []
    if len(crit_id) > 0:
        crit_tokens = tokenizer.convert_ids_to_tokens(crit_id, skip_special_tokens=True)
    if len(non_crit_id) > 0:
        non_crit_tokens = tokenizer.convert_ids_to_tokens(non_crit_id, skip_special_tokens=True)

    if len(non_crit_tokens) == 0 and len(crit_tokens) == 0:
        crit_tokens = tokenizer.convert_ids_to_tokens(id_list, skip_special_tokens=True)

    dictionary = {
    "input_text": in_txt,
    "target_text": tar_txt,
    "initial_suffix": emoji.demojize(suf_txt),
    "Frequency": freq_info,
    "critical_tokens": crit_tokens,
    "non_critical_tokens": non_crit_tokens,
    "CT_sim_target": critical_sim_list[1],
    "CT_sim_input": critical_sim_list[2],
    "CT_closeto_target": ct_closeto_target,
    "critical_token_combination_index": crit_idx,
    "number_of_critical_tokens": len(crit_id),
    "remove_non_critical": match
    }

    json_object = json.dumps(dictionary, indent=4)
    with open(dir_path+"summary.json", "w") as outfile:
        outfile.write(json_object)

    return len(crit_tokens), len(non_crit_tokens), match, tokenizer.decode(crit_id, skip_special_tokens=True)

In [ ]:
def call_main():

    input_target_data = []
    with open("noun_attacks.jsonl", "r") as f:
        for line in f:
            _json = json.loads(line)
            input_target_data.append((_json["input_text"],_json["target_text"], _json["suffix"], _json["tag"], _json["seed"]))

    df = {}
    for inp, tar, suf, tag, seed in input_target_data:
        inp = inp.lower()
        tar = tar.lower()
        suf = emoji.emojize(suf)
        seed = int(seed)
        tag = tag.lower()

        crit_len, non_crit_len, match, crit_tokens = main_job(inp, tar, suf, seed, 49407)

        print(crit_len, non_crit_len, match)
        print(crit_tokens)
        print(f"Prompt pairs done of {tag} POS!!!")

        df['input'] = inp
        df['target'] = tar
        df['suffix'] = suf
        df['tag'] = tag
        df['seed'] = seed
        df['crit_len'] = crit_len
        df['non_crit_len'] = non_crit_len
        df['remove'] = match
        df['crit_tokens'] = crit_tokens

        with open('finalized_noun_stats.jsonl', 'a') as fp:
          json.dump(df, fp)
          fp.write("\n")

call_main()

In [ ]:
def critical_comb(inp_txt, tar_txt, sufx, non_cric, seed, dir_path):

    sufx_toks = tokenizer.tokenize(sufx)
    sufx_ids = tokenizer.convert_tokens_to_ids(sufx_toks)
    non_cric_ids = tokenizer.convert_tokens_to_ids(non_cric)

    inp_img = generate_image_no_show(inp_txt, seed)
    tar_img = generate_image_no_show(tar_txt, seed)

    critical_checklist = []
    img_match = []

    # Define the placeholder
    placeholder = 49407

    # Find the indices in the large list where the small list elements are located
    indices_to_replace = [i for i, x in enumerate(sufx_ids) if x not in non_cric_ids]

    # Generate all possible combinations of these indices
    for r in range(1, len(indices_to_replace)):
        for comb in combinations(indices_to_replace, r):
            new_list = sufx_ids[:]
            for index in comb:
                new_list[index] = placeholder
            critical_checklist.append(new_list)

#     print(len(critical_checklist))
    path = dir_path+"critic_removed/"
    make_directory(path)
    dnt = 0
    candidate_list, text_shift_list, image_shift_list = [], [], []

    for idx, crit in enumerate(critical_checklist):
        sfx = tokenizer.decode(crit, skip_special_tokens=True)
        crit_text = inp_txt + sfx
        crit_img = generate_image_no_show(crit_text, seed, path+f"{idx}", True)

        #in_succ = check_attack_success(crit_img, inp_txt)
        crit_succ = check_attack_success(crit_img, tar_txt)

        text_sim_a = calculate_text_embedding_shift(crit_text, inp_txt)
        text_sim_b = calculate_text_embedding_shift(crit_text, tar_txt)

        image_sim_a = calculate_image_embedding_shift(crit_img, inp_img)
        image_sim_b = calculate_image_embedding_shift(crit_img, tar_img)

        candidate_list.append(sfx)
        text_shift_list.append(text_sim_b - text_sim_a)
        image_shift_list.append(image_sim_b - image_sim_a)

        if crit_succ == 0:
            dnt += 1
        else:
            img_match.append(idx)

    not_match = False
    if len(img_match) == 0:
        not_match = True

    df = pd.DataFrame()
    df['candidate_suffix'] = candidate_list
    df['text_shift'] = text_shift_list
    df['img_shift'] = image_shift_list
    df.to_csv(path+"suffix_evaluation.csv", index=False)

    crit_tokens_ids = [x for x in sufx_ids if x not in non_cric_ids]
    crit_tokens = []
    if len(crit_tokens_ids) > 0:
        crit_tokens = tokenizer.convert_ids_to_tokens(crit_tokens_ids, skip_special_tokens=True)

    diction = {
        "tokens": crit_tokens,
        "average text shift": sum(text_shift_list) / len(text_shift_list),
        "average image shift": sum(image_shift_list) / len(image_shift_list),
        "total image": len(critical_checklist),
        "images not matched": dnt,
        "images matched": len(critical_checklist) - dnt,
        "images match id": img_match,
        "simply_remove": not_match
    }
    json_object = json.dumps(diction, indent=4)
    with open(path+"info.json", "w") as outfile:
        outfile.write(json_object)

    return sum(text_shift_list) / len(text_shift_list), sum(image_shift_list) / len(image_shift_list)

In [ ]:
def critical_input_prompts():

    input_target_data = []
    with open("noun_attacks.jsonl", "r") as f:
        for line in f:
            _json = json.loads(line)
            input_target_data.append((_json["input_text"],_json["target_text"], _json["suffix"], _json["tag"], _json["seed"], _json["non_cric"]))

    t_shift, i_shift = [], []
    for inp, tar, suf, tag, seed, non_cric in input_target_data:
        inp = inp.lower()
        tar = tar.lower()
        suf = emoji.emojize(suf)
        seed = int(seed)
        tag = tag.lower()
        non_cric_toks=non_cric

        print(inp, tar, non_cric_toks)

        dir_path = inp.replace(" ","_").replace(".","_")+"___"+tar.replace(" ","_").replace(".","_")+"/"
        dir_path = "./" + dir_path
        make_directory(dir_path)

        t, i = critical_comb(inp, tar, suf, non_cric_toks, seed, dir_path)
        t_shift.append(t)
        i_shift.append(i)

        print(f"Prompt pairs done of {tag} POS!!!")

    df = pd.DataFrame()
    df['avg_text_sift'] = t_shift
    df['avg_image_shift'] = i_shift
    df.to_csv('verb_shifts.csv', index=False)

critical_input_prompts()